##### Group the data into weekends and workdays first. Because from the time series data, the traffic flow on the weekend of July 6 and July 7 is relatively stable and less traffic flow, other weekend days are the same, need to be trained separately for modeling. The characteristics of traffic flow data are significantly different during weekends and weekdays, so ARIMA models are established separately.

Nodes 4689-4662 are taken as the main nodes for analysis. The spatial adjacency matrix is constructed. The path directly connected with the road is the first-order adjacency matrix, and the path separated by two segments is the second-order adjacency matrix.

In [ ]:
from statsmodels.tsa.arima_model import ARMA

road_data = df[df['DEVICEID'] == 5696].copy()
road_data['FROMTIME'] = pd.to_datetime(road_data['FROMTIME'])
road_data.index = road_data['FROMTIME']
road_data['FROMTIME-Time'] = road_data['FROMTIME'].apply(findTime)  # 找到时间
road_data['FROMTIME-Date'] = road_data['FROMTIME'].apply(findDate)  # 找到日期
xianquan = road_data.resample(datetime.timedelta(seconds=5 * 60)).sum()
df = xianquan.copy()

# Select the data from 7:00 to 22:00 as required by the problem
df['FROMTIME'] = pd.to_datetime(df.index)
df.index=df['FROMTIME']
df['FROMTIME-Time'] = df["FROMTIME"].apply(findTime)  # find time
df['FROMTIME-Date'] = df["FROMTIME"].apply(findDate)  # find date
df=df[(pd.to_datetime(df["FROMTIME-Time"],format = '%H:%M:%S')>= pd.to_datetime('07:00:00',format = '%H:%M:%S'))&(pd.to_datetime(df["FROMTIME-Time"],format = '%H:%M:%S')<= pd.to_datetime('22:00:00',format = '%H:%M:%S'))]


In [ ]:
# df1 is for weekdays and df2 is for weekends
df1=df[(pd.to_datetime(df["FROMTIME-Date"],format = '%Y-%m-%d')>= pd.to_datetime('2019-07-03',format = '%Y-%m-%d'))&(pd.to_datetime(df["FROMTIME-Date"],format = '%Y-%m-%d')<= pd.to_datetime('2019-07-05',format = '%Y-%m-%d'))]
df2=df[(pd.to_datetime(df["FROMTIME-Date"],format = '%Y-%m-%d')>= pd.to_datetime('2019-07-06',format = '%Y-%m-%d'))&(pd.to_datetime(df["FROMTIME-Date"],format = '%Y-%m-%d')<= pd.to_datetime('2019-07-07',format = '%Y-%m-%d'))]

In [ ]:
# overall data for coil
import matplotlib.dates as mdates
fig=plt.figure(figsize= (20,5))
ax = fig.add_subplot(1,1,1)  
plt.plot(df.index,df['FLOW'])
plt.title(str('5696'))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.xticks(rotation=90)

In [ ]:
fig=plt.figure(figsize= (20,5))
ax = fig.add_subplot(1,1,1)  
plt.plot(df1.index,df1['FLOW'])
plt.plot(df2.index,df2['FLOW'])
plt.title(str('5696'))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
plt.xticks(rotation=90)

#### Check whether the data is stable

In [ ]:
# Looking at the data according to the plot, the visual sensation data has a certain seasonal
df1["FLOW"].plot()

Unit root stationary test: The statistical properties of a given sequence (mean, variance, and covariance) are not constants of time, which is a prerequisite for a stationary time series. The variance of a stationary time series cannot be a function of time. The unit root test ** checks for the presence of a unit root in the sequence by checking the value of a=1 **. The following are the two most commonly used unit root stationary test methods: 1. Dickey Fuller test (ADF test); 2.

In [ ]:
'''
Perform ADF test 
Return value of adf_test 
Test statistic: indicates the test statistics 
p-value: indicates the probability of a P-value test 
Lags used: Lags are automatically selected when k, autolag=AIC is used 
Number of Observations Used: Number of samples 
Critical Value(5%) : A critical value with a significance level of 5%. 
(1) It is assumed that there is a unit root, that is, instability; 
(2) Significance level, 1% : strict rejection of null hypothesis; 5% : Reject null hypothesis, 10% by analogy. 
(3) Look at the size of P-value and significance level a, the smaller the P-value is, if it is less than the significance level, the null hypothesis is rejected and the series is considered to be stable; If it is greater than that, it cannot be rejected as unstable 
(4) Look at the test statistics and critical values, if the test statistics are less than the critical value, we reject the null hypothesis and think that the series is stable; If it is greater than that, it cannot be rejected as unstable
'''
## ADF test
from statsmodels.tsa.stattools import adfuller
def adf_test(timeseries):
#     rolling_statistics(timeseries) # plot
    print ('Results of Augment Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value   # Increase the critical value of the significance level behind
    print (dfoutput)
    
adf_test(df["FLOW"])   # It can be seen from the results that the P-value is very small, rejecting the null hypothesis and thinking that it is a stationary sequence

In [ ]:
# White noise test
'''
The acorr_ljungbox(x, lags=None, boxpierce=False) function checks for no autocorrelation 
lags represent the number of delay periods, if it is an integer, then the number of delay periods included, and if it is a list or array, then all lags are included in the largest lag in the list 
boxpierce True indicates that in addition to LB statistics, Box and Pierce's Q statistics are also returned 
Return value: 
lbvalue: statistics of the test
bpvalue:((optionsal), float or array) – test statistic for Box-Pierce test
bppvalue:((optional), float or array) – p-value based for Box-Pierce test on chi-square distribution
'''
from statsmodels.stats.diagnostic import acorr_ljungbox
def test_stochastic(ts,lag):
    p_value = acorr_ljungbox(ts, lags=lag) #lags can be self-defined
    return p_value


test_stochastic(df["FLOW"],[6,12])# If p-value is less than 0.05, then the null hypothesis can be rejected as not a white noise sequence

#### Determine the order of ARMA

In [ ]:
#### Autocorrelation graph ACF and partial correlation graph PACF
import statsmodels.api as sm
def acf_pacf_plot(ts_log_diff):
    sm.graphics.tsa.plot_acf(ts_log_diff,lags=40) #ARIMA,q
    sm.graphics.tsa.plot_pacf(ts_log_diff,lags=40) #ARIMA,p
    
acf_pacf_plot(df["FLOW"])  

##### This is a cyclical data, one cycle per day. That is, the interval is 15 hours (15*12=180), and there are 181 data a day from 7:00 to 22:00

In [ ]:
df.index[543]

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1= fig.add_subplot(111)
# Make first-order difference and re-difference for the same period data
diff1= df1["FLOW"].diff(181).diff(1)
diff1=diff1.dropna()
acf_pacf_plot(diff1)
diff1.plot(ax=ax1)
# After the first-order difference, the stationarity test has been carried out. It can be seen from the ACF and PACF diagrams that ACF8 order is truncated and PACF8 order is truncated. ARIMA (8,1,8)
adf_test(diff1)

#### Modeling

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(diff1,order=(8,0,6)) 
results_AR = model.fit(disp=-1)
prediction=results_AR.fittedvalues
plt.plot(diff1)
plt.plot(prediction, color='red')
sum((prediction[2:]-diff1[2:])**2)/diff1.size

In [ ]:
forecast= results_AR.predict(start =363, end= 724, dynamic= True)  
df3=df[(pd.to_datetime(df["FROMTIME-Date"],format = '%Y-%m-%d')>= pd.to_datetime('2019-07-13',format = '%Y-%m-%d'))&(pd.to_datetime(df["FROMTIME-Date"],format = '%Y-%m-%d')<= pd.to_datetime('2019-07-14',format = '%Y-%m-%d'))]
df3.plot()
forecast.plot()

In [ ]:
predict_ts = results_AR.predict()

diff_shift_ts = diff1.shift(1)
diff_recover_1 = predict_ts.add(diff_shift_ts)

In [ ]:
diff_recover_1.plot(color='blue', label='Predict')
df["FLOW"].plot(color='red', label='Original')
plt.legend(loc='best')
plt.title('RMSE: %.4f'% np.sqrt(sum((diff_recover_1-df["FLOW"])**2)/df["FLOW"].size))
plt.show()

In [ ]:
diff1= df["FLOW"].diff(1).diff(1)
diff1=diff1.dropna()
model = ARIMA(df["FLOW"], order=(0, 2, 1))  
results_AR = model.fit(disp=-1) 
plt.plot(diff1)
prediction=results_AR.fittedvalues
plt.plot(prediction, color='green')
plt.title('RSS: %.4f'% sum((prediction[2:]-diff1[2:])**2))

In [ ]:
prediction_restored = pd.Series([time_series[0]], index=[time_series.index[0]]) .append(time_series_diff).cumsum()

In [ ]:
diff1= df["FLOW"].diff(2)
model = ARIMA(df["FLOW"], order=(0, 2, 1))  
results_ARIMA = model.fit(disp=-1)  
plt.plot(diff1)
plt.plot(results_MA.fittedvalues, color='green')
plt.title('RSS: %.4f'% sum((results_MA.fittedvalues-diff1[2:])**2))

In [ ]:
diff_shift_ts = diff1.shift(1)
diff_recover_1 = results_MA.fittedvalues.add(diff_shift_ts)

rol_sum = ts_log.rolling(window=1).sum()
rol_recover = diff_recover*12 - rol_sum.shift(1)

rol_recover.plot(color='blue', label='Predict')
df["FLOW"].plot(color='red', label='Original')
plt.legend(loc='best')
plt.title('RMSE: %.4f'% np.sqrt(sum((rol_recover-df["FLOW"])**2)/ts.size))
plt.show()

In [ ]:
df1=df
plt.plot(df["FLOW"])
plt.plot(results_AR.fittedvalues, color='red') 

In [ ]:
# decompose

def decompose(timeseries):

    # return contains three part: trend， seasonal and residual
    decomposition = seasonal_decompose(timeseries,freq=156)
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid
    
    plt.subplot(411)
    plt.plot(df["FLOW"], label='Original')
    plt.legend(loc='best')
    plt.subplot(412)
    plt.plot(trend, label='Trend')
    plt.legend(loc='best')
    plt.subplot(413)
    plt.plot(seasonal,label='Seasonality')
    plt.legend(loc='best')
    plt.subplot(414)
    plt.plot(residual, label='Residuals')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()
    return trend , seasonal, residual
trend , seasonal, residual = decompose(df["FLOW"])
residual.dropna(inplace=True)

In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import pandas as pd
import numpy as np
import datetime
import os
import itertools

In [ ]:
def findTime(x):
    return x.time()
def findDate(x):
    return x.date()


filelist = os.listdir('measurement')
filelist.sort()
startpoint = np.where(np.array(filelist) == '16157')[0][0]
filelist = filelist[startpoint:]

df = pd.DataFrame()
for fileid in filelist:
    data = pd.read_csv('measurement/'+fileid+'/tab_ldt.csv')
    df = pd.concat([df,data])


for roadid in df['DEVICEID'].unique(): 
    road_data = df[df['DEVICEID'] == roadid].copy()
    road_data['FROMTIME'] = pd.to_datetime(road_data['FROMTIME'])
    road_data.index = road_data['FROMTIME']
    road_data['FROMTIME-Time'] = road_data['FROMTIME'].apply(findTime) 
    road_data['FROMTIME-Date'] = road_data['FROMTIME'].apply(findDate) 
    road_data.to_csv('train_data/data/'+'%s.csv'%roadid)
    plt.figure(figsize= (20,5))
    plt.title(str(roadid))
    for date in road_data['FROMTIME-Date'].unique():
        df_temp = road_data[road_data['FROMTIME-Date'] == date]
        data3 = df_temp.resample(datetime.timedelta(seconds=5 * 60)).sum()
        data3.index = pd.Series(data3.index).apply(findTime)
        plt.plot(data3.index,data3['FLOW'],label=date)
    plt.legend()